In [14]:
import bs4 as bs
import requests

In [15]:
sauce = requests.get('http://www.soprasteria.in/footer/sitemap').text

In [16]:
soup = bs.BeautifulSoup(sauce,'html.parser')

In [17]:
main_list = soup.find_all('ul')[5]

In [27]:
links_to_parse = []
for link in main_list.find_all('a'):
    links_to_parse.append("http://www.soprasteria.in" + link.get('href')[2:])


In [28]:
for pag in links_to_parse:
    mSauce = requests.get(pag).text
    mSoup = bs.BeautifulSoup(mSauce,'lxml')
    fh = open("%s.txt" % pag.split('/')[-1],"w")
    for t in mSoup.body.find_all('p'):
        fh.write(t.text.encode('utf-8'))
        fh.write('\n\n')
    fh.close()

In [37]:
sauce = requests.get('http://www.soprasteria.in/about-us/locations').text
soup = bs.BeautifulSoup(sauce,'lxml')

In [47]:
text = soup.find('div', attrs={ "class" : "content main"}).text.encode('utf-8')
fh = open("locations.txt","w")
fh.write(text)
fh.close()

In [110]:
sauce = requests.get('https://www.soprasteria.com/en/markets/aerospace').text
soup = bs.BeautifulSoup(sauce,'lxml')

for script in soup(["script", "style"]):
    script.decompose()    # rip it out

fh = open("markets.txt","w")
text = soup.find('div', attrs={ "class" : "brick _red _width_x2 _height_mini _nudge_bottom _nudge_right_alpha _right"}).text.encode('utf-8')
fh.write(text)
text = soup.find('div', attrs={ "class" : "sf_colsOut content main"}).text.encode('utf-8')

fh.write(text)
fh.close()

In [149]:
sauce = requests.get('https://www.soprasteria.com/en/markets').text
soup = bs.BeautifulSoup(sauce,'lxml')

for script in soup(["script", "style"]):
    script.decompose()    # rip it out

text = soup.find('div', attrs={ "id" : "placeholder_3_widget_0_C001_Col00"}).text.encode('utf-8')

lines = (line.strip() for line in text.splitlines())

text = '\n'.join(line for line in lines if line)

#fh = open("markets.txt","w")
#fh.write(text)
#fh.close()

markets_to_crawl = []

for market in soup.find('div', attrs={ "class" : "brick _alpha_light _right"}).findAll('a'):
    markets_to_crawl.append("https://www.soprasteria.com" + market.get('href').encode('utf-8'))


In [151]:
for link in markets_to_crawl:
    sauce = requests.get(link).text
    soup = bs.BeautifulSoup(sauce,'lxml')

    for script in soup(["script", "style"]):
        script.decompose()    # rip it out

    fh = open("%s.txt" % link.split('/')[-1],"w")
    text = soup.find('div', attrs={ "class" : "brick _red _width_x2 _height_mini _nudge_bottom _nudge_right_alpha _right"}).text.encode('utf-8')
    fh.write(text)
    text = soup.find('div', attrs={ "class" : "sf_colsOut content main"}).text.encode('utf-8')

    fh.write(text)
    fh.close()

In [196]:
pages_left = ['http://www.soprasteria.in/about-us/global-delivery-model',
              'http://www.soprasteria.in/careers/life-in-sopra-steria'
             ]

for link in pages_left:
    sauce = requests.get(link).text
    soup = bs.BeautifulSoup(sauce,'lxml')

    for script in soup(["script", "style"]):
        script.decompose()
        
    text = soup.find('div', attrs={ "class" : "sf_colsIn sf_2cols_1in_67"}).text.encode('utf-8')
    fh = open("%s.txt" % link.split('/')[-1],"w")
    fh.write(text)
    fh.close()

In [230]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium import webdriver

driver = webdriver.PhantomJS()
driver.get("https://steria.taleo.net/careersection/in_cs_ext_fs/jobsearch.ftl?lang=en&radiusType=K&location=462170431401&searchExpanded=true&radius=1")
table = wait(driver, 10).until(EC.presence_of_element_located(("xpath", "//table[@id='jobs' and ./tbody/tr]")))
text1 = table.text
next_button = driver.find_element_by_link_text("Next")
next_button.click()

wait(driver, 5).until(lambda x: next_button.get_attribute("aria-disabled") == "true")
table = wait(driver, 10).until(EC.presence_of_element_located(("xpath", "//table[@id='jobs' and ./tbody/tr]")))
text2 = table.text
driver.close()

In [233]:
fh = open('current-openings.txt','w')
fh.write(text1.encode('utf-8'))
fh.write(text2.encode('utf-8'))
fh.close()

In [281]:
sauce = requests.get('https://www.soprasteria.com/en/solutions').text
soup = bs.BeautifulSoup(sauce,'lxml')

solution_links = []
i=0
    
for link in soup.find('div', attrs={ "class" : "brick _alpha_light _right"}).find_all('a'):
    if i != 5:
        solution_links.append("http://www.soprasteria.com" + link.get('href'))
    i=i+1

In [318]:
for link in solution_links:
    while True:
        try:
            sauce = requests.get(link).text
        except requests.ConnectionError:
            continue
        break
    soup = bs.BeautifulSoup(sauce,'lxml')
    
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    
    
    if soup.find('div', attrs={ "class" : "brick _alpha_light _right"}) is None:
        text = soup.find('div', attrs={ "class" : "brick _red _width_x2 _height_mini _nudge_bottom _nudge_right_alpha _right"}).text.encode('utf-8')
        fh = open("%s.txt" % link.split('/')[-1],"w")
        fh.write(text)
        text = soup.find('div', attrs={ "class" : "sf_colsIn sf_2cols_1in_67"}).text.encode('utf-8')
        fh.write(text)
        fh.close()
    else:
        text = soup.find('div', attrs={ "class" : "brick _red _width_x2 _nudge_bottom _right"}).text.encode('utf-8')
        fh = open("%s.txt" % link.split('/')[-1],"w")
        fh.write(text)
        text = soup.find('div', attrs={ "class" : "sf_colsIn sf_2cols_1in_67"}).text.encode('utf-8')
        fh.write(text)
        fh.close()
        for l in soup.find('div', attrs={ "class" : "brick _alpha_light _right"}).find_all('a'):
            page = "http://www.soprasteria.com" + l.get('href')
            while True:
                try:
                    s = requests.get(page).text
                except requests.ConnectionError:
                    continue
                break
            su = bs.BeautifulSoup(s,'lxml')
            text = su.find('div', attrs={ "class" : "brick _red _width_x2 _height_mini _nudge_bottom _nudge_right_alpha _right"}).text.encode('utf-8')
            fh = open("%s.txt" % page.split('/')[-1],"w")
            fh.write(text)
            text = su.find('div', attrs={ "class" : "sf_colsIn sf_2cols_1in_67"}).text.encode('utf-8')
            fh.write(text)
            fh.close()
            
            

link
link
link
link
link
link


In [302]:
TODO: https://www.soprasteria.com/en/offerings

consulting
systems-integration
scientific-technical-industrial-and-embedded-software-engineering
applications-management
testing
infrastructure-management
business-process-services
software
